In [1]:
import rsa
from binascii import hexlify
import numpy as np

In [2]:
# Generate RSA key pair (public key and private key)
# You can adjust the key size as needed
(public_key, private_key) = rsa.newkeys(128)

# Message to be encrypted
message = b"hello"

# Encrypt the message with the public key
encrypted_message = rsa.encrypt(message, public_key)

# Decrypt the message with the private key
decrypted_message = rsa.decrypt(encrypted_message, private_key)

# Print the results
print("Original message:", message.decode('utf-8'))
print("Encrypted message:", encrypted_message)
print("Decrypted message:", decrypted_message.decode('utf-8'))

Original message: hello
Encrypted message: b"\x13\x8a\x18\xf6\xdc\x00\x1d\xe0\\'[\xbd\x9f \x0c\xc9"
Decrypted message: hello


In [3]:
def hex_to_bits(hex_string):
    # Convert hex to binary
    binary_representation = bin(int(hex_string, 16))[2:]

    # Ensure leading zeros are included
    length = len(hex_string) * 4  # Each hex digit represents 4 bits
    return binary_representation.zfill(length)


def string_to_bits(input_string):
    return ''.join(format(ord(char), '08b') for char in input_string)

# dataset

In [4]:
import nltk
from nltk.corpus import genesis

genesis_words = genesis.words()


def filter_letter(let):

    if ord('a') <= ord(let) <= ord('z'):
        return True

    else:
        return False
    

def make_blocks(word_list, block_size):
    word_str = "".join(word_list)
    new_str = ""
    blocked_arr = []

    for i in range(0, len(word_str)):

        let = word_str[i].lower()
        if filter_letter(let):
            new_str += let
        else:
            pass

    del word_str

    for i in range(0, len(new_str), block_size):
        block = new_str[i:i+block_size]

        if len(block) == block_size:
            blocked_arr.append(block)
        else:
            diff = block_size - len(block)
            padded_block = block + "x" * diff
            blocked_arr.append(padded_block)

    return blocked_arr

In [5]:
pt_blocks = make_blocks(genesis_words, 1)

In [6]:
pt_blocks[0]

'i'

In [7]:
X = []

for pt in pt_blocks:
    t = string_to_bits(pt)
    X.append([int(i) for i in list(t)])



In [8]:
X = np.array(X)

In [9]:
X.shape

(1074706, 8)

# Autoencoder

In [10]:
from keras.models import Sequential
from keras.layers import Dense

# Define the autoencoder model
autoencoder = Sequential()

# Encoder
autoencoder.add(Dense(10, input_shape=(8,), activation='relu'))  # Hidden layer
# Output layer (encoded representation)
autoencoder.add(Dense(100, activation='relu'))

# Decoder
autoencoder.add(Dense(100, activation='relu'))  # Hidden layer
# Output layer (reconstructed input)
autoencoder.add(Dense(8, activation='sigmoid'))

# Print the summary of the autoencoder model
autoencoder.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                90        
                                                                 
 dense_1 (Dense)             (None, 100)               1100      
                                                                 
 dense_2 (Dense)             (None, 100)               10100     
                                                                 
 dense_3 (Dense)             (None, 8)                 808       
                                                                 
Total params: 12098 (47.26 KB)
Trainable params: 12098 (47.26 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [11]:

# Compile the autoencoder model
# Using Mean Squared Error as the loss function
autoencoder.compile(optimizer='adam', loss='mse',metrics=['accuracy'])

# Assuming you have your input data, let's say 'X_train'
# Replace 'X_train' with your actual training data

# Train the autoencoder
autoencoder.fit(X, X, epochs=50, batch_size=32,
                shuffle=True, validation_split=0.2)

# Evaluate the autoencoder on the training data
loss = autoencoder.evaluate(X, X)
print(f"Training Loss: {loss}")

# You can also get predictions from the autoencoder
encoded_data = autoencoder.predict(X)


Epoch 1/50


26868/26868 [==============================] - 31s 1ms/step - loss: 3.7438e-04 - accuracy: 0.1917 - val_loss: 1.1065e-10 - val_accuracy: 0.1771
Epoch 2/50
 5439/26868 [=====>........................] - ETA: 21s - loss: 8.7363e-11 - accuracy: 0.2112

KeyboardInterrupt: 